In [4]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [5]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invasive_ssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 63s 55ms/step - loss: 157.0787 - accuracy: 0.4077 - val_loss: 29.1292 - val_accuracy: 0.4146
Epoch 2/100
985/985 [==============================] - 50s 51ms/step - loss: 21.4489 - accuracy: 0.4315 - val_loss: 12.7934 - val_accuracy: 0.3812
Epoch 3/100
985/985 [==============================] - 54s 55ms/step - loss: 7.3389 - accuracy: 0.4578 - val_loss: 8.3072 - val_accuracy: 0.5475
Epoch 4/100
502/985 [==============>...............] - ETA: 25s - loss: 18.2942 - accuracy: 0.4483

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.32534717986352407
Test Accuracy: 0.3678651228925452


In [33]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 40s 36ms/step - loss: 0.8048 - accuracy: 0.7427 - val_loss: 3.4905 - val_accuracy: 0.5420
Epoch 2/100
985/985 [==============================] - 38s 38ms/step - loss: 0.9070 - accuracy: 0.6952 - val_loss: 1.4212 - val_accuracy: 0.5050
Epoch 3/100
985/985 [==============================] - 46s 47ms/step - loss: 0.6569 - accuracy: 0.7597 - val_loss: 0.6924 - val_accuracy: 0.7569
Epoch 4/100
985/985 [==============================] - 38s 39ms/step - loss: 0.6276 - accuracy: 0.7750 - val_loss: 1.9463 - val_accuracy: 0.5611
Epoch 5/100
985/985 [==============================] - 43s 43ms/step - loss: 0.5192 - accuracy: 0.8143 - val_loss: 1.4517 - val_accuracy: 0.7217
Epoch 6/100
985/985 [==============================] - 42s 42ms/step - loss: 0.4746 - accuracy: 0.8305 - val_loss: 4.0191 - val_accuracy: 0.4836
Epoch 7/100
985/985 [==============================] - 38s 39ms/step - loss: 0.6172 - accuracy: 0.7769 - val_loss: 0.8028 - val_ac

Epoch 56/100
985/985 [==============================] - 40s 40ms/step - loss: 0.1103 - accuracy: 0.9617 - val_loss: 0.7177 - val_accuracy: 0.8455
Epoch 57/100
985/985 [==============================] - 36s 36ms/step - loss: 0.1036 - accuracy: 0.9639 - val_loss: 0.5247 - val_accuracy: 0.8563
Epoch 58/100
985/985 [==============================] - 42s 43ms/step - loss: 0.1514 - accuracy: 0.9490 - val_loss: 0.5352 - val_accuracy: 0.8215
Epoch 59/100
985/985 [==============================] - 41s 41ms/step - loss: 0.1058 - accuracy: 0.9642 - val_loss: 0.3347 - val_accuracy: 0.8960
Epoch 60/100
985/985 [==============================] - 38s 39ms/step - loss: 0.1278 - accuracy: 0.9545 - val_loss: 0.3406 - val_accuracy: 0.8889
Epoch 61/100
985/985 [==============================] - 38s 38ms/step - loss: 0.1085 - accuracy: 0.9616 - val_loss: 0.3665 - val_accuracy: 0.8788
Epoch 62/100
985/985 [==============================] - 37s 38ms/step - loss: 0.0925 - accuracy: 0.9677 - val_loss: 0.3664 -

In [28]:
from sklearn.metrics import accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6181190331098924


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)